In [1]:
import numpy as np
from scipy.spatial import ConvexHull as ch
from scipy.spatial import Delaunay

from coxeter.families import (
    ArchimedeanFamily as archfam,
)
from coxeter.families import (
    JohnsonFamily as johnfam,
)
from coxeter.families import (
    PlatonicFamily as platfam,
)
from coxeter.shapes import ConvexPolyhedron

In [2]:
cube = platfam.get_shape("Cube")
# tetr = platfam.get_shape("Tetrahedron")
# mbri = johnfam.get_shape("Metabidiminished Rhombicosidodecahedron")
# icosi = archfam.get_shape("Icosidodecahedron")
# poly = cube

In [3]:
mbri.faces.__len__(), mbri._coplanar_simplices.__len__()

(42, 42)

In [3]:
hull = ch(mbri.vertices, qhull_options="Q2")
# hull.equations
# hull.neighbors

In [249]:
def _find_coplanar_simplices(self, rounding: int = 15):
    """
    Get lists of simplex indices for coplanar simplices.

    Args:
        rounding (int, optional):
            Integer number of decimal places to round equations to.
            (Default value: 15).


    """
    # Combine simplex indices
    equation_groups = defaultdict(list)

    # Iterate over all simplex equations
    for i, equation in enumerate(self._simplex_equations):
        # Convert equation into hashable tuple
        equation_key = tuple(equation.round(rounding))
        equation_groups[equation_key].append(i)
    ragged_coplanar_indices = [
        np.fromiter(set(group), np.int32) for group in equation_groups.values()
    ]

    self._coplanar_simplices = ragged_coplanar_indices

In [4]:
# hull.simplices

In [5]:
# First - compute normals of each triangle
# triangle_edges = np.dstack([hull.simplices[:, :-1], hull.simplices[:, 1:]])
print(hull.simplices[:, :-1][0], hull.simplices[:, 1:][0])
triangle_edges = np.dstack([hull.simplices[:, :-1], hull.simplices[:, 1:]]).transpose(
    0, 2, 1
)
# %timeit -r 10 -n 100000
display(triangle_edges[0])
triangle_edge_endpoints = hull.points[triangle_edges]
triangle_edge_endpoints.shape, triangle_edge_endpoints[0]

[48 40] [40 38]


array([[48, 40],
       [40, 38]], dtype=int32)

((96, 2, 2, 3),
 array([[[ 0.54304647,  0.        , -0.39295212],
         [ 0.63580989,  0.15009435, -0.15009435]],
 
        [[ 0.63580989,  0.15009435, -0.15009435],
         [ 0.63580989, -0.15009435, -0.15009435]]]))

In [130]:
# hull.points

In [131]:
# poly._coplanar_simplices

In [132]:
triangle_edge_vectors = (
    triangle_edge_endpoints[..., 0, :] - triangle_edge_endpoints[..., 1, :]
)
# np.diff(triangle_edge_endpoints, axis=2).squeeze()

# triangle_edge_vectors.shape, triangle_edge_vectors

In [133]:
normals = np.cross(triangle_edge_vectors[:, 1, :], triangle_edge_vectors[:, 0, :])
normals / np.linalg.norm(normals, axis=1)[:, None]
nnormals = normals / np.linalg.norm(normals, axis=1)[:, None]
nnormals[1::2] *= -1
# nnormals
# I think there is something about the sorting of triangles from convexhull that result in
# odd triangles having opposite orientations in the first half, and evens having opposite in the second

In [134]:
# ok - so I can just use this.
# hull.equations[:, :3]

In [135]:
# 0 element of array is current simplex. next trhee are neighbors
simplex_indices = np.hstack(
    [np.arange(0, hull.neighbors.shape[0], 1)[:, None], hull.neighbors]
)
simplex_indices

array([[ 0,  6,  1,  8],
       [ 1,  2,  0,  4],
       [ 2,  1,  3,  4],
       [ 3,  7,  2, 10],
       [ 4,  1,  2,  5],
       [ 5,  9, 11,  4],
       [ 6,  0,  7,  8],
       [ 7,  3,  6, 10],
       [ 8,  0,  6,  9],
       [ 9,  5, 11,  8],
       [10,  3,  7, 11],
       [11,  5,  9, 10]])

In [6]:
hull_normals = hull.equations[:, :3]
# hull_normals[simplex_indices]
coplanar_simplex_array = np.all(
    hull_normals[np.arange(0, hull.nsimplex, 1)][:, None]
    == hull_normals[hull.neighbors],
    axis=2,
)

# This is right! We get the correct coplanar simplices. How do we index back in?
coplanar_simplex_array

array([[False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False,  True,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [False,  True, False],
       [False,  True, False],
       [False,  True, False],
       [False,  True, False],
       [False, False,  True],
       [False,  True, False],
       [False,  True,  True],
       [False, False,  True],
       [False, False,  True],
       [False, False,  True],
       [Fa

In [7]:
hull.neighbors[
    coplanar_simplex_array
]  # Direct indexing returns a flattened list - not useful
[
    [*hull.neighbors[i][coplanar_simplex_array[i]], i] for i in range(hull.nsimplex)
].__len__()

96

In [8]:
# What does the above do?
for i in range(hull.nsimplex):
    print([*hull.neighbors[i][coplanar_simplex_array[i]], i])
    # first, we get the coplanar simplices and heighbors for simplex i
    # then, index in the coplanar neighbors and dump them into an array
    # we then add i, as the indexed simplex is also coplanar

[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
[11, 10]
[10, 11]
[13, 12]
[12, 13]
[15, 14]
[16, 14, 15]
[15, 16]
[18, 17]
[17, 18]
[20, 19]
[19, 20]
[22, 21]
[21, 22]
[24, 23]
[23, 24]
[26, 25]
[25, 26]
[29, 27]
[29, 28]
[28, 27, 29]
[31, 30]
[30, 31]
[34, 32]
[34, 33]
[33, 32, 34]
[36, 35]
[35, 36]
[38, 39, 37]
[37, 38]
[37, 39]
[41, 40]
[40, 41]
[43, 42]
[42, 43]
[46, 44]
[46, 45]
[44, 45, 46]
[48, 47]
[47, 48]
[50, 49]
[49, 51, 50]
[50, 51]
[53, 52]
[52, 53]
[55, 54]
[54, 55]
[58, 56]
[58, 57]
[56, 57, 58]
[60, 59]
[59, 60]
[62, 63, 61]
[61, 62]
[61, 63]
[65, 64]
[64, 65]
[67, 66]
[66, 67]
[69, 68]
[68, 69]
[76, 70]
[73, 77, 71]
[74, 72]
[71, 76, 73]
[75, 72, 74]
[74, 77, 75]
[70, 73, 76]
[75, 71, 77]
[79, 78]
[78, 80, 79]
[79, 80]
[82, 81]
[81, 82]
[90, 88, 83]
[86, 88, 84]
[89, 85]
[84, 87, 86]
[89, 86, 87]
[84, 83, 88]
[87, 85, 89]
[83, 90]
[93, 91]
[93, 92]
[91, 92, 93]
[95, 94]
[94, 95]


In [9]:
func = lambda x: tuple(sorted(x))
arr = np.random.rand(1000)
func(arr)

(0.0017318337087434132,
 0.0020163124217437156,
 0.0035447480247632335,
 0.00394259414847542,
 0.006609278010620279,
 0.00853404435978633,
 0.009010762298194974,
 0.009899284064141756,
 0.010115961801518636,
 0.012404473627683599,
 0.012447484935732667,
 0.012974861536379279,
 0.013309849813057273,
 0.01387163413332293,
 0.014703931931117853,
 0.014808475350076455,
 0.018428864919364796,
 0.018672799765176995,
 0.019334288844548464,
 0.021070590262666555,
 0.02110502873856468,
 0.02129166678763994,
 0.021528721783741545,
 0.021807986622800035,
 0.022052899647635127,
 0.02208917208035044,
 0.023289882081194424,
 0.024778974093295214,
 0.024975308660231677,
 0.029590825098955165,
 0.031197292029556123,
 0.031204465611203602,
 0.033693294520900086,
 0.03415956062156722,
 0.035767986888116554,
 0.03622266558646581,
 0.036248777345704974,
 0.03717957413822137,
 0.03824200878882533,
 0.039367799521233926,
 0.041576934354527806,
 0.04162211069230026,
 0.04232261923436931,
 0.04279335787077321

In [10]:
%timeit -r 10 -n 1000 [[*hull.neighbors[i][coplanar_simplex_array[i]], i] for i in range(hull.nsimplex)]

210 µs ± 32.5 µs per loop (mean ± std. dev. of 10 runs, 1,000 loops each)


In [180]:
# There can be a variable number of neighbors! If one triangle has multiple coplanar simplices,
# an array indexing will not work. A list comprehension is probably fine here
# Take the coplanar neighbors for each simplex and add the current simplex (see above)
coplanar_simplices = [
    {*hull.neighbors[i][coplanar_simplex_array[i]], i} for i in range(hull.nsimplex)
]
coplanar_simplices

[{0},
 {1},
 {2},
 {3},
 {4},
 {5},
 {6},
 {7},
 {8},
 {9},
 {10, 11},
 {10, 11},
 {12, 13},
 {12, 13},
 {14, 15},
 {14, 15, 16},
 {15, 16},
 {17, 18},
 {17, 18},
 {19, 20},
 {19, 20},
 {21, 22},
 {21, 22},
 {23, 24},
 {23, 24},
 {25, 26},
 {25, 26},
 {27, 29},
 {28, 29},
 {27, 28, 29},
 {30, 31},
 {30, 31},
 {32, 34},
 {33, 34},
 {32, 33, 34},
 {35, 36},
 {35, 36},
 {37, 38, 39},
 {37, 38},
 {37, 39},
 {40, 41},
 {40, 41},
 {42, 43},
 {42, 43},
 {44, 46},
 {45, 46},
 {44, 45, 46},
 {47, 48},
 {47, 48},
 {49, 50},
 {49, 50, 51},
 {50, 51},
 {52, 53},
 {52, 53},
 {54, 55},
 {54, 55},
 {56, 58},
 {57, 58},
 {56, 57, 58},
 {59, 60},
 {59, 60},
 {61, 62, 63},
 {61, 62},
 {61, 63},
 {64, 65},
 {64, 65},
 {66, 67},
 {66, 67},
 {68, 69},
 {68, 69},
 {70, 76},
 {71, 73, 77},
 {72, 74},
 {71, 73, 76},
 {72, 74, 75},
 {74, 75, 77},
 {70, 73, 76},
 {71, 75, 77},
 {78, 79},
 {78, 79, 80},
 {79, 80},
 {81, 82},
 {81, 82},
 {83, 88, 90},
 {84, 86, 88},
 {85, 89},
 {84, 86, 87},
 {86, 87, 89},
 {83, 

In [185]:
{0, 1, 2}.issuperset({1, 2})
# for test_set in coplanar_simplices:
# What if we cech subset and superset?
for test_set in coplanar_simplices:
    for other_set in coplanar_simplices:
        pass  #
    # Or can we do j>i?

In [177]:
# coplanar_simplices[20]#coplanar_simplices[21]
def is_subset(s, sets_list):
    # return any(s.issubset(other_set) for other_set in sets_list if s != other_set)
    return any(s.issubset(other_set) for other_set in sets_list if s != other_set)

In [179]:
# %%timeit -r 10 -n 1000
# %timeit -r 10 -n 1000 coplanar_simplices[20].issubset(coplanar_simplices[21])
filtered_sets = [s for s in coplanar_simplices if not is_subset(s, coplanar_simplices)]
# filtered_sets

In [45]:
%timeit -r 10 -n 10000 poly._find_coplanar_simplices()

40.3 µs ± 102 ns per loop (mean ± std. dev. of 10 runs, 10,000 loops each)


In [100]:
is_subset({0, 1}, [{0}, {0, 1}, {0, 1, 2}])

True

In [108]:
{0, 1}.issubset({0, 1})

True

In [109]:
i = 0
[*hull.neighbors[i][coplanar_simplices[i]], i].sort()
sorted([*hull.neighbors[i][coplanar_simplices[i]], i])

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
np.arange(hull.nsimplex), coplanar_simplices.T

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55]),
 array([[False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False],
        [False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False,  True,  True, False,  True,  True,  True,
          True, False, False,  True,  True,  True, False, False,  True

In [ ]:
hull.neighbors

array([[24, 54, 45],
       [22, 30, 37],
       [41, 36, 29],
       [20, 39, 27],
       [21, 31, 26],
       [22, 35, 47],
       [23, 40, 46],
       [44, 32, 55],
       [50, 28, 25],
       [38, 23, 27],
       [42, 51, 29],
       [28, 52, 31],
       [43, 36, 34],
       [48, 33, 35],
       [20, 39, 49],
       [34, 43, 53],
       [33, 48, 44],
       [40, 46, 49],
       [25, 54, 50],
       [42, 51, 53],
       [ 3, 14, 21],
       [ 4, 22, 20],
       [ 1, 21,  5],
       [ 6,  9, 24],
       [ 0, 25, 23],
       [18, 24,  8],
       [ 4, 27, 28],
       [ 3, 26,  9],
       [11,  8, 26],
       [ 2, 10, 30],
       [ 1, 31, 29],
       [ 4, 30, 11],
       [ 7, 33, 34],
       [16, 13, 32],
       [15, 12, 32],
       [ 5, 37, 13],
       [ 2, 12, 37],
       [ 1, 35, 36],
       [ 9, 40, 39],
       [ 3, 14, 38],
       [ 6, 38, 17],
       [ 2, 43, 42],
       [10, 19, 41],
       [12, 41, 15],
       [ 7, 16, 45],
       [ 0, 46, 44],
       [ 6, 45, 17],
       [ 5, 4

In [59]:
# This was close - but I'm thinking about this the wrong way. All we need is equations
normals = hull.equations[:, :3]
normals

[tuple(normal) for normal in normals]

[(-0.0, -0.0, -1.0),
 (-0.0, -0.0, -1.0),
 (0.0, -1.0, 0.0),
 (0.0, -1.0, 0.0),
 (1.0, -0.0, -0.0),
 (1.0, -0.0, -0.0),
 (-1.0, -0.0, -0.0),
 (-1.0, -0.0, -0.0),
 (0.0, 1.0, -0.0),
 (0.0, 1.0, -0.0),
 (-0.0, -0.0, 1.0),
 (-0.0, -0.0, 1.0)]

[[ True  True False False False False False False False False False False]
 [ True  True False False False False False False False False False False]
 [False False  True  True False False False False False False False False]
 [False False  True  True False False False False False False False False]
 [False False False False  True  True False False False False False False]
 [False False False False  True  True False False False False False False]
 [False False False False False False  True  True False False False False]
 [False False False False False False  True  True False False False False]
 [False False False False False False False False  True  True False False]
 [False False False False False False False False  True  True False False]
 [False False False False False False False False False False  True  True]
 [False False False False False False False False False False  True  True]]


In [15]:
mbri.get_face_area().__len__(), len(mbri.faces)
face_areas = np.array(mbri.get_face_area())

In [23]:
sum(face_areas)

2.7821089295582024

In [14]:
mbri.surface_area

5.1295458308488735

In [31]:
mbri._coplanar_simplices.__len__()

62

In [32]:
mbri.faces.__len__()

42

In [58]:
np.unique([item for row in mbri._coplanar_simplices for item in row])
# So each item in the coplanar simplices is unique

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95], dtype=int32)

In [71]:
cs = np.array(mbri._coplanar_simplices, dtype=object)
cs[0:20]

array([array([0], dtype=int32), array([1], dtype=int32),
       array([2], dtype=int32), array([3], dtype=int32),
       array([4], dtype=int32), array([5], dtype=int32),
       array([6], dtype=int32), array([7], dtype=int32),
       array([8], dtype=int32), array([9], dtype=int32),
       array([10, 11], dtype=int32), array([12], dtype=int32),
       array([13], dtype=int32), array([14], dtype=int32),
       array([16, 15], dtype=int32), array([17, 18], dtype=int32),
       array([19], dtype=int32), array([20], dtype=int32),
       array([21], dtype=int32), array([22], dtype=int32)], dtype=object)

In [73]:
mbri.faces[0:20]

[array([48, 40, 38], dtype=int32),
 array([24, 14, 22], dtype=int32),
 array([25, 23, 15], dtype=int32),
 array([ 2,  7, 10], dtype=int32),
 array([8, 6, 0], dtype=int32),
 array([41, 49, 39], dtype=int32),
 array([33, 36, 45], dtype=int32),
 array([17, 29, 20], dtype=int32),
 array([32, 43, 34], dtype=int32),
 array([16, 18, 27], dtype=int32),
 array([25, 15, 30, 21], dtype=int32),
 array([48, 45, 36, 40], dtype=int32),
 array([ 1,  3, 30, 15, 28], dtype=int32),
 array([24, 22, 23, 25], dtype=int32),
 array([19, 28, 15, 23], dtype=int32),
 array([48, 38, 34, 43], dtype=int32),
 array([ 3,  1,  9, 11], dtype=int32),
 array([18, 22, 14, 27], dtype=int32),
 array([18, 26, 19, 23, 22], dtype=int32),
 array([17,  7,  2, 29], dtype=int32)]

In [13]:
len(mbri._coplanar_simplices), len(mbri.faces)

(62, 42)

In [71]:
mbri._coplanar_simplices

[array([0], dtype=int32),
 array([1], dtype=int32),
 array([2], dtype=int32),
 array([3], dtype=int32),
 array([4], dtype=int32),
 array([5], dtype=int32),
 array([6], dtype=int32),
 array([7], dtype=int32),
 array([8], dtype=int32),
 array([9], dtype=int32),
 array([10, 11], dtype=int32),
 array([12], dtype=int32),
 array([13], dtype=int32),
 array([14], dtype=int32),
 array([16, 15], dtype=int32),
 array([17, 18], dtype=int32),
 array([19], dtype=int32),
 array([20], dtype=int32),
 array([21], dtype=int32),
 array([22], dtype=int32),
 array([24, 23], dtype=int32),
 array([25], dtype=int32),
 array([26], dtype=int32),
 array([27, 28, 29], dtype=int32),
 array([30, 31], dtype=int32),
 array([32, 33, 34], dtype=int32),
 array([35, 36], dtype=int32),
 array([37], dtype=int32),
 array([38, 39], dtype=int32),
 array([40], dtype=int32),
 array([41], dtype=int32),
 array([42, 43], dtype=int32),
 array([44, 45, 46], dtype=int32),
 array([47], dtype=int32),
 array([48], dtype=int32),
 array([4

In [73]:
%%timeit -r 10 -n 1000
# Start with cube
snorms = mbri._simplex_equations[:, :3]
tol = 1e-15

coplanar_simplices = [() for i in range(len(snorms))]
for i, simplex_normal in enumerate(snorms):
    coplanars = np.all(np.abs(simplex_normal - snorms) < tol, axis=1)
    for ind, co in enumerate(coplanars):
        # Iterating means each tuple will have the same length - nice!
        # print(i, co)
        if co:
            coplanar_simplices[i] += (ind,)
list(set(coplanar_simplices))

1.12 ms ± 11.8 µs per loop (mean ± std. dev. of 10 runs, 1,000 loops each)


In [77]:
%%timeit -r 10 -n 1000
mbri._find_coplanar_simplices()

345 µs ± 11.8 µs per loop (mean ± std. dev. of 10 runs, 1,000 loops each)


In [112]:
%%timeit -r 10 -n 10000
snorms = mbri._simplex_equations[:, :3]
tol = 1e-15
coplanars = np.all(np.abs(snorms[:, None] - snorms) < tol, axis=2)
coplanar_simplices = list({tuple(np.where(co)[0]) for co in coplanars})
coplanar_simplices

515 µs ± 3.72 µs per loop (mean ± std. dev. of 10 runs, 10,000 loops each)


In [113]:
%%timeit -r 10 -n 10000
mbri._find_coplanar_simplices()

338 µs ± 9.85 µs per loop (mean ± std. dev. of 10 runs, 10,000 loops each)


In [140]:
snorms = mbri._simplex_equations[:, :3]
tol = 1e-15
coplanars = np.all(np.abs(snorms[:, None] - snorms) < tol, axis=2)
coplanar_simplices = list({tuple(np.where(co)[0]) for co in coplanars})

In [142]:
for co in coplanars:
    # print(co)
    # display(np.where(co)[0])
    %timeit -r 10 -n 100000 np.where(co)[0]
    cp = np.where(co)[0]
    %timeit -r 10 -n 100000 tuple(cp)
    %timeit -r 10 -n 100000 np.arange(0, len(mbri.simplices))[co]
    break

788 ns ± 111 ns per loop (mean ± std. dev. of 10 runs, 100,000 loops each)
474 ns ± 44.5 ns per loop (mean ± std. dev. of 10 runs, 100,000 loops each)
1 µs ± 21.1 ns per loop (mean ± std. dev. of 10 runs, 100,000 loops each)


In [145]:
%timeit -r 10 -n 1000 coplanars = np.all(np.abs(snorms[:, None] - snorms) < tol, axis=2)

456 µs ± 115 µs per loop (mean ± std. dev. of 10 runs, 1,000 loops each)


In [230]:
snorms = mbri._simplex_equations[:, :3]
coplanars = np.all(np.abs(snorms[:, None] - snorms) < tol, axis=2)
coplanar_simplices = list({tuple(np.where(co)[0]) for co in coplanars})
print(coplanars)

[[ True False False ... False False False]
 [False  True False ... False False False]
 [False False  True ... False False False]
 ...
 [False False False ...  True False False]
 [False False False ... False  True  True]
 [False False False ... False  True  True]]


In [257]:
# %%timeit -r 10 -n 1000
snorms = mbri._simplex_equations[:, :3]
coplanars = np.all(np.abs(snorms[:, None] - snorms) < tol, axis=2)
coplanar_simplices = [tuple(np.where(co)[0]) for co in coplanars]
coplanar_simplices

[(0,),
 (1,),
 (2,),
 (3,),
 (4,),
 (5,),
 (6,),
 (7,),
 (8,),
 (9,),
 (10, 11),
 (10, 11),
 (12, 13),
 (12, 13),
 (14, 15, 16),
 (14, 15, 16),
 (14, 15, 16),
 (17, 18),
 (17, 18),
 (19, 20),
 (19, 20),
 (21, 22),
 (21, 22),
 (23, 24),
 (23, 24),
 (25, 26),
 (25, 26),
 (27, 28, 29),
 (27, 28, 29),
 (27, 28, 29),
 (30, 31),
 (30, 31),
 (32, 33, 34),
 (32, 33, 34),
 (32, 33, 34),
 (35, 36),
 (35, 36),
 (37, 38, 39),
 (37, 38, 39),
 (37, 38, 39),
 (40, 41),
 (40, 41),
 (42, 43),
 (42, 43),
 (44, 45, 46),
 (44, 45, 46),
 (44, 45, 46),
 (47, 48),
 (47, 48),
 (49, 50, 51),
 (49, 50, 51),
 (49, 50, 51),
 (52, 53),
 (52, 53),
 (54, 55),
 (54, 55),
 (56, 57, 58),
 (56, 57, 58),
 (56, 57, 58),
 (59, 60),
 (59, 60),
 (61, 62, 63),
 (61, 62, 63),
 (61, 62, 63),
 (64, 65),
 (64, 65),
 (66, 67),
 (66, 67),
 (68, 69),
 (68, 69),
 (70, 71, 72, 73, 74, 75, 76, 77),
 (70, 71, 72, 73, 74, 75, 76, 77),
 (70, 71, 72, 73, 74, 75, 76, 77),
 (70, 71, 72, 73, 74, 75, 76, 77),
 (70, 71, 72, 73, 74, 75, 76, 77),

In [252]:
# %%timeit -r 10 -n 1000
# np.argwhere(coplanars)
snorms = mbri._simplex_equations[:, :3]
coplanars = np.all(np.abs(snorms[:, None] - snorms) < tol, axis=2)
writeto = [() for _ in range(len(coplanars))]
for i, val in np.argwhere(coplanars):
    writeto[i] += (val,)
output = set(writeto)
# output

In [6]:
from collections import defaultdict

equation_groups = defaultdict(list)
eqution_keys = []
# Iterate over all simplex equations
for i, equation in enumerate(mbri._simplex_equations):
    # Convert to hashable key
    equation_key = tuple(equation.round(15))

    # Store vertex indices from the new simplex under the correct equation key
    equation_groups[equation_key].extend(mbri._simplices[i])
    eqution_keys.append(equation_key)
# Combine elements with the same plan equation and remove duplicate indices
ragged_faces = [np.fromiter(set(group), np.int32) for group in equation_groups.values()]
print(set(eqution_keys).__len__())

62


In [7]:
x = [tuple(row) for row in cube._simplex_equations]
x

[(0.0, 0.0, -1.0, -0.5),
 (0.0, 0.0, -1.0, -0.5),
 (0.0, -1.0, -0.0, -0.5),
 (-0.0, -1.0, 0.0, -0.5),
 (1.0, 0.0, 0.0, -0.5),
 (1.0, -0.0, 0.0, -0.5),
 (-1.0, 0.0, 0.0, -0.5),
 (-1.0, -0.0, 0.0, -0.5),
 (0.0, 1.0, 0.0, -0.5),
 (0.0, 1.0, -0.0, -0.5),
 (0.0, 0.0, 1.0, -0.5),
 (0.0, -0.0, 1.0, -0.5)]

In [15]:
snorms = cube._simplex_equations[:, :3]
tol = 1e-15
coplanars = np.all(np.abs(snorms[:, None] - snorms) < tol, axis=2)
coplanar_simplices = list({tuple(np.where(co)[0]) for co in coplanars})
coplanar_simplices
# So this gets me an array of coplanar simplices. Now, how do I tie back to the faces?

[(0, 1), (10, 11), (2, 3), (6, 7), (4, 5), (8, 9)]

In [19]:
cube._simplex_areas = cube._find_triangle_array_area(
    cube._vertices[cube._simplices], sum_result=False
)
cube._simplex_areas

array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])

In [21]:
cube._simplex_areas[cube._coplanar_simplices[0]]

TypeError: 'dict_values' object is not subscriptable

In [22]:
cube._coplanar_simplices

dict_values([[0, 1], [2, 3], [4, 5], [6, 7], [8, 9], [10, 11]])

In [30]:
hash(mbri._simplex_equations[10].round(15).tobytes()), hash(
    mbri._simplex_equations[11].round(15).tobytes()
)

(6409598020480052466, 6409598020480052466)

In [4]:
%timeit -r 10 -n 10000 mbri._find_coplanar_simplices()
%timeit -r 10 -n 10000 mbri._combine_simplices()

178 µs ± 2.04 µs per loop (mean ± std. dev. of 10 runs, 10,000 loops each)
263 µs ± 226 ns per loop (mean ± std. dev. of 10 runs, 10,000 loops each)


In [9]:
mbri.faces[0:10]

[array([48, 40, 38], dtype=int32),
 array([24, 14, 22], dtype=int32),
 array([25, 23, 15], dtype=int32),
 array([ 2,  7, 10], dtype=int32),
 array([8, 6, 0], dtype=int32),
 array([41, 49, 39], dtype=int32),
 array([33, 36, 45], dtype=int32),
 array([17, 29, 20], dtype=int32),
 array([32, 43, 34], dtype=int32),
 array([16, 18, 27], dtype=int32)]

In [10]:
mbri.simplices[0:10]

array([[38, 48, 40],
       [14, 22, 24],
       [25, 23, 15],
       [ 2,  7, 10],
       [ 8,  6,  0],
       [49, 39, 41],
       [36, 45, 33],
       [17, 29, 20],
       [32, 43, 34],
       [18, 27, 16]], dtype=int32)

In [6]:
# So sort_faces never changes the order of faces in the list
# So as long as combine_simplices and find_coplanar_simplices
# return the same order, we should be ok!
%timeit -r 10 -n 1000 mbri._combine_simplices()

265 µs ± 5.58 µs per loop (mean ± std. dev. of 10 runs, 1,000 loops each)


In [16]:
# %%timeit -r 10 -n 10000
poly = mbri
simplex_normals = poly._simplex_equations[:, :3]
tol = 1e-15
coplanars = np.all(np.abs(simplex_normals[:, None] - simplex_normals) < tol, axis=2)
coplanar_simplices = list({tuple(np.where(co)[0]) for co in coplanars})
# coplanar_simplices[0:10]
# now - use coplanar simplices to build faces
# faces = [
#    np.array(list(set(poly.simplices[[co]].flatten()))) for co in coplanar_simplices
# ]

# Move the simplices out into an array that matches the new sorting
# ordered_simplices = [simplex for coplanar in coplanar_simplices for simplex in coplanar]

array([10, 20, 10,  2])

In [74]:
%%timeit -r 10 -n 10000
poly = mbri
simplex_normals = poly._simplex_equations[:, :3]
tol = 1e-15
# Generate boolean array checking which simplices share a normal (within tolerance)
is_coplanar = np.all(np.abs(simplex_normals[:, None] - simplex_normals) < tol, axis=2)
# Convert boolean array into ragged list of coplanar simplices
coplanar_simplices = list({tuple(np.where(coplanar)[0]) for coplanar in is_coplanar})
faces = []
ordered_simplices = []
for face_simplex_indices in coplanar_simplices:
    # Get unique vertex indices from faces and convert to np array
    faces.append(
        np.fromiter(set(poly.simplices[[face_simplex_indices]].flat), np.int32)
    )
    # Add simplex indices back into list that matches face ordering
    ordered_simplices.extend(face_simplex_indices)
    # faces.append(set(poly.simplices[[face_simplex_indices]].flatten()))
# faces.__len__(), coplanar_simplices.__len__(), ordered_simplices.__len__()

651 µs ± 26.5 µs per loop (mean ± std. dev. of 10 runs, 10,000 loops each)


In [82]:
poly._simplex_equations
poly._simplex_areas

AttributeError: 'ConvexPolyhedron' object has no attribute '_simplex_areas'

In [63]:
np.sum(np.array([len(face) for face in faces])[:, None] == [3, 4, 5, 10], axis=0)
# so we get the right number of vertices and faces!

array([10, 20, 10,  2])

In [91]:
i = 3
poly._simplex_equations[coplanar_simplices[i][0]]

array([ 0.        ,  0.        , -1.        , -0.63580989])

In [96]:
poly._simplex_equations.round(5)[3]

array([ 0.     ,  0.35682, -0.93417, -0.64751])

In [66]:
%%timeit -r 10 -n 10000
mbri._combine_simplices()
mbri._find_coplanar_simplices()

710 µs ± 14.2 µs per loop (mean ± std. dev. of 10 runs, 10,000 loops each)


In [26]:
%%timeit -r 10 -n 10000
faces = [
    np.array(list(set(poly.simplices[[co]].flatten()))) for co in coplanar_simplices
]

179 µs ± 1.85 µs per loop (mean ± std. dev. of 10 runs, 10,000 loops each)


In [6]:
%%timeit -r 10 -n 10000
mbri._combine_simplices()
mbri._find_coplanar_simplices()

706 µs ± 6.3 µs per loop (mean ± std. dev. of 10 runs, 10,000 loops each)


In [96]:
%timeit -r 10 -n 100000 [simplex for coplanar in coplanar_simplices for simplex in coplanar]
%timeit -r 10 -n 100000 list(sum(coplanar_simplices,()))

3.52 µs ± 229 ns per loop (mean ± std. dev. of 10 runs, 100,000 loops each)
7.24 µs ± 463 ns per loop (mean ± std. dev. of 10 runs, 100,000 loops each)


In [30]:
t1 = mbri.vertices[mbri.simplices[81]]
np.cross(
    np.diff(t1[[0, 1], :], axis=0).squeeze(), np.diff(t1[[1, 2], :], axis=0).squeeze()
)

array([-0.04505663,  0.07290316, -0.02784653])

In [31]:
np.diff(t1[[0, 1], :], axis=0).squeeze()

array([ 0.24285777,  0.09276341, -0.15009435])

In [42]:
nor = np.cross(t1[1, :] - t1[0, :], t1[2, :] - t1[0, :])
nor /= np.linalg.norm(nor)
nor  # So the normal is right!

array([-0.5       ,  0.80901699, -0.30901699])

In [38]:
mbri._simplex_equations[81]

array([-0.5       ,  0.80901699, -0.30901699, -0.63580989])

In [43]:
# Somehow, we get the correct number and hsape of faces but different than we expect?